In [1]:
import time
import math
import warnings
import numpy as np
import pandas as pd
import lightgbm as lgb
import itertools
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)
lg=2

In [2]:
#读取数据
demand_train = pd.read_csv('G:/kaggle竞赛GM之路！！！/竞赛ing/2021工业大数据竞赛-物料需求预测/data_final/demand_train.csv')

In [3]:
def prepare(data):
	
	# 重命名
	data = data.rename(columns={'过账日期':'date','需求量':'label','工厂编码':'f_code','物料编码':'m_code','物料品牌':'brand','物料类型':'cat_f','物料品类':'cat_son'})
	
	# 拆分日期，编码物料需求所属月份
	data['date'] = pd.to_datetime(data['date'])
	data['day'] = data['date'].apply(lambda x : x.day)
	data['month'] = data['date'].apply(lambda x : x.month)
	data['year'] = data['date'].apply(lambda x : x.year)
	
	#如果日期在12月24日之后，则划为下一年1月；如果日期在1月-11月的24日之后，则划为下一月
	
	# 月份+1
	data['month_id'] = list(map(lambda x,y:y+1 if (x>24)and(y==12) else y,data['day'],data['month']))
	# 年+1
	data['sales_year'] = list(map(lambda y,z:z+1 if y==13 else z,data['month_id'],data['year']))
	# 再把13月变为1月
	data['month_id'] = list(map(lambda y:1 if y==13 else y,data['month_id']))
	
	# 编码time_id
	data['time_id'] = list(map(lambda x,y:(x-2018)*12+y,data['sales_year'],data['month_id']))
	
	# 删除多余特征
	data.drop(['day','month','year','date'],axis=1,inplace=True)
	
	# 合并f_code和m_code生成唯一id
	data['id'] = data['f_code'].astype(str)+data['m_code'].astype(str)
	
	# 合并每个月的物料需求
	data = data.groupby(['id','f_code','m_code','brand','cat_f','cat_son','sales_year','month_id','time_id']).agg({'label':'sum'}).reset_index()
	
	# 笛卡尔积id, time_id
	d_all = pd.DataFrame(itertools.product(list(set(data['id'])),list(set(data['time_id']))),columns=['id','time_id'])
	d_all_pred = pd.DataFrame(itertools.product(list(set(data['id'])),[39,40,41]),columns=['id','time_id'])
	
	# 拼接'f_code','m_code','brand','cat_f','cat_son'
	d_pivot = pd.pivot_table(data,index=['id','f_code','m_code','brand','cat_f','cat_son'])
	d_c = pd.DataFrame(d_pivot).reset_index()[['id','f_code','m_code','brand','cat_f','cat_son']]
	d_all = pd.merge(d_all,d_c,on=['id'],how='left')
	d_all_pred = pd.merge(d_all_pred,d_c,on=['id'],how='left')
	
	# 拼接'sales_year','month_id'
	m_pivot = pd.pivot_table(data,index=['time_id','sales_year','month_id'])
	d_m = pd.DataFrame(m_pivot).reset_index()[['time_id','sales_year','month_id']]
	d_all = pd.merge(d_all,d_m,on=['time_id'],how='left')
	#决赛需要修改map
	d_all_pred['sales_year'] = d_all_pred['time_id'].map({39: 2021, 40: 2021, 41: 2021})
	d_all_pred['month_id'] = d_all_pred['time_id'].map({39: 3, 40: 4, 41: 5})
	
	# 拼接
	data = pd.merge(d_all,data,on=['id','time_id','f_code','m_code','brand','cat_f','cat_son','sales_year','month_id'],how='left').sort_values(by=['id','time_id'],ascending=True).reset_index(drop=True)
	
	# 用0填充所有需求缺失数据
	data.fillna(0, inplace=True)
	
	# 拼接
	data = pd.concat([data,d_all_pred]).sort_values(by=['time_id','id'],ascending=True).reset_index(drop=True)
	
	return data

In [4]:
input_data = prepare(demand_train)

In [5]:
input_data['demandVolume'] = input_data['label']

In [6]:
def get_stat_feature(df_,month):   
	data = df_.copy()
	stat_feat = []
	#循环预测
	#start = 0
	#直接预测
	start = month-38
	
	#'历史月销量' 
	for last in range(1,13):
		tmp=data.copy()
		tmp['time_id'] = list(map(lambda x:x+last+start if x+last+start<=41 else -1,tmp['time_id']))
		tmp = tmp[~tmp['time_id'].isin([-1])][['label','time_id','id']]
		tmp = tmp.rename(columns={'label':'last_{0}_sale'.format(last)})
		data = pd.merge(data,tmp,how='left',on=['time_id','id'])
		stat_feat.append('last_{0}_sale'.format(last)) 
	
	# id强相关特征
	#1-3个月，4-6个月
	data['1_3_sum']  = data.loc[:,'last_1_sale':'last_3_sale'].sum(1)
	data['4_6_sum']  = data.loc[:,'last_4_sale':'last_6_sale'].sum(1)
	data['1_3_mean'] = data.loc[:,'last_1_sale':'last_3_sale'].mean(1)
	data['4_6_mean'] = data.loc[:,'last_4_sale':'last_6_sale'].mean(1)
	#1-6个月
	data['1_6_sum'] = data.loc[:,'last_1_sale':'last_6_sale'].sum(1)
	data['1_6_mean'] = data.loc[:,'last_1_sale':'last_6_sale'].mean(1)
	data['1_6_max'] = data.loc[:,'last_1_sale':'last_6_sale'].max(1)
	data['1_6_min'] = data.loc[:,'last_1_sale':'last_6_sale'].min(1)
	data['1_6_bias'] = data['last_1_sale'] - data['1_6_mean']
	#1-12个月
	data['1_12_sum'] = data.loc[:,'last_1_sale':'last_12_sale'].sum(1)
	data['1_12_mean'] = data.loc[:,'last_1_sale':'last_12_sale'].mean(1)
	data['1_12_max'] = data.loc[:,'last_1_sale':'last_12_sale'].max(1)
	data['1_12_min'] = data.loc[:,'last_1_sale':'last_12_sale'].min(1)
	data['1_12_bias'] = data['last_1_sale'] - data['1_12_mean']
	#6-12个月
	data['6_12_sum']  = data.loc[:,'last_6_sale':'last_12_sale'].sum(1)
	data['6_12_mean']  = data.loc[:,'last_6_sale':'last_12_sale'].mean(1)
	
	sales_stat_feat_3 = ['1_3_sum','4_6_sum','1_3_mean','4_6_mean']
	sales_stat_feat_6 = ['1_6_sum','1_6_mean','1_6_max','1_6_min','1_6_bias']
	sales_stat_feat_12 = ['1_12_sum','1_12_mean','1_12_max','1_12_min','1_12_bias','6_12_sum','6_12_mean']
	stat_feat = stat_feat + sales_stat_feat_3 + sales_stat_feat_6 + sales_stat_feat_12
	
	
	corr_list = ['last_1_sale','last_3_sale','last_6_sale','last_12_sale','1_3_sum','4_6_sum','1_6_sum','1_12_sum','6_12_sum']
	
	# f_code强相关特征
	f_code_feat = []
	for col in corr_list:
		pivot = pd.pivot_table(data,index=['f_code','time_id'],values=col,aggfunc=np.sum)
		pivot = pd.DataFrame(pivot).rename(columns={col:col+'_f_code'}).reset_index()
		data  = pd.merge(data,pivot,on=['f_code','time_id'],how='left')
		f_code_feat.append(col+'_f_code')
	
	data['1_6_bias_f_code'] = data['last_1_sale_f_code']/data['1_6_sum_f_code']
	data['1_12_bias_f_code'] = data['last_1_sale_f_code']/data['1_12_sum_f_code']
	stat_feat = stat_feat + f_code_feat + ['1_6_bias_f_code','1_12_bias_f_code']
	
	# m_code强相关特征
	m_code_feat = []
	for col in corr_list:
		pivot = pd.pivot_table(data,index=['m_code','time_id'],values=col,aggfunc=np.sum)
		pivot = pd.DataFrame(pivot).rename(columns={col:col+'_m_code'}).reset_index()
		data  = pd.merge(data,pivot,on=['m_code','time_id'],how='left')
		m_code_feat.append(col+'_m_code')
	
	data['1_6_bias_m_code'] = data['last_1_sale_m_code']/data['1_6_sum_m_code']
	data['1_12_bias_m_code'] = data['last_1_sale_m_code']/data['1_12_sum_m_code']
	stat_feat = stat_feat + m_code_feat + ['1_6_bias_m_code','1_12_bias_m_code']
	
	# brand强相关特征
	brand_feat = []
	for col in corr_list:
		pivot = pd.pivot_table(data,index=['brand','time_id'],values=col,aggfunc=np.sum)
		pivot = pd.DataFrame(pivot).rename(columns={col:col+'_brand'}).reset_index()
		data  = pd.merge(data,pivot,on=['brand','time_id'],how='left')
		brand_feat.append(col+'_brand')
	
	data['1_6_bias_brand'] = data['last_1_sale_brand']/data['1_6_sum_brand']
	data['1_12_bias_brand'] = data['last_1_sale_brand']/data['1_12_sum_brand']
	stat_feat = stat_feat + brand_feat + ['1_6_bias_brand','1_12_bias_brand']
	
	
	# cat_f强相关特征
	cat_f_feat = []
	for col in corr_list:
		pivot = pd.pivot_table(data,index=['cat_f','time_id'],values=col,aggfunc=np.sum)
		pivot = pd.DataFrame(pivot).rename(columns={col:col+'_cat_f'}).reset_index()
		data  = pd.merge(data,pivot,on=['cat_f','time_id'],how='left')
		cat_f_feat.append(col+'_cat_f')
	
	data['1_6_bias_cat_f'] = data['last_1_sale_cat_f']/data['1_6_sum_cat_f']
	data['1_12_bias_cat_f'] = data['last_1_sale_cat_f']/data['1_12_sum_cat_f']
	stat_feat = stat_feat + cat_f_feat + ['1_6_bias_cat_f','1_12_bias_cat_f']
	
	
	#cat_son强相关特征
	cat_son_feat = []
	for col in corr_list:
		pivot = pd.pivot_table(data,index=['cat_son','time_id'],values=col,aggfunc=np.sum)
		pivot = pd.DataFrame(pivot).rename(columns={col:col+'_cat_son'}).reset_index()
		data  = pd.merge(data,pivot,on=['cat_son','time_id'],how='left')
		cat_son_feat.append(col+'_cat_son')
	
	data['1_6_bias_cat_son'] = data['last_1_sale_cat_son']/data['1_6_sum_cat_son']
	data['1_12_bias_cat_son'] = data['last_1_sale_cat_son']/data['1_12_sum_cat_son']
	stat_feat = stat_feat + cat_son_feat + ['1_6_bias_cat_son','1_12_bias_cat_son']
	
	
	# id趋势特征
	data['1_2_diff'] = data['last_1_sale'] - data['last_2_sale']
	data['1_3_diff'] = data['last_1_sale'] - data['last_3_sale']
	data['1_4_diff'] = data['last_1_sale'] - data['last_4_sale']
	data['1_7_diff'] = data['last_1_sale'] - data['last_7_sale']
	data['2_3_diff'] = data['last_2_sale'] - data['last_3_sale']
	data['2_4_diff'] = data['last_2_sale'] - data['last_4_sale']
	data['3_4_diff'] = data['last_3_sale'] - data['last_4_sale']
	data['3_6_diff'] = data['last_3_sale'] - data['last_6_sale']
	data['6_12_diff'] = data['last_6_sale'] - data['last_12_sale']
	data['jidu_13_46_diff'] = data['1_3_sum'] - data['4_6_sum']
	data['hy_16_612_diff'] = data['1_6_sum'] - data['6_12_sum']
	
	trend_stat_feat_1 = ['1_2_diff','1_3_diff','1_4_diff','1_7_diff']
	trend_stat_feat_2 = ['2_3_diff','2_4_diff']
	trend_stat_feat_3 = ['3_4_diff','3_6_diff']
	trend_stat_feat_6 = ['6_12_diff']
	trend_stat_feat_jidu = ['jidu_13_46_diff']
	trend_stat_feat_hy = ['hy_16_612_diff']
	stat_feat = stat_feat + trend_stat_feat_1 + trend_stat_feat_2 + trend_stat_feat_3 + trend_stat_feat_6
	stat_feat = stat_feat + trend_stat_feat_jidu + trend_stat_feat_hy
	
	
	tend_list = ['1_2_diff','1_3_diff','1_4_diff','1_7_diff','2_3_diff','2_4_diff','3_4_diff','3_6_diff','6_12_diff',
				'jidu_13_46_diff','hy_16_612_diff']
	
	# f_code趋势
	f_code_feat = []
	for col in tend_list:
		pivot = pd.pivot_table(data,index=['f_code','time_id'],values=col,aggfunc=np.sum)
		pivot = pd.DataFrame(pivot).rename(columns={col:col+'_f_code'}).reset_index()
		data  = pd.merge(data,pivot,on=['f_code','time_id'],how='left')
		f_code_feat.append(col+'_f_code')
	
	stat_feat = stat_feat + f_code_feat
	
	
	# m_code趋势
	m_code_feat = []
	for col in tend_list:
		pivot = pd.pivot_table(data,index=['m_code','time_id'],values=col,aggfunc=np.sum)
		pivot = pd.DataFrame(pivot).rename(columns={col:col+'_m_code'}).reset_index()
		data  = pd.merge(data,pivot,on=['m_code','time_id'],how='left')
		m_code_feat.append(col+'_m_code')
	
	stat_feat = stat_feat + m_code_feat
	
	
	# brand趋势
	brand_feat = []
	for col in tend_list:
		pivot = pd.pivot_table(data,index=['brand','time_id'],values=col,aggfunc=np.sum)
		pivot = pd.DataFrame(pivot).rename(columns={col:col+'_brand'}).reset_index()
		data  = pd.merge(data,pivot,on=['brand','time_id'],how='left')
		brand_feat.append(col+'_brand')
	
	stat_feat = stat_feat + brand_feat
	
	
	# cat_f趋势
	cat_f_feat = []
	for col in tend_list:
		pivot = pd.pivot_table(data,index=['cat_f','time_id'],values=col,aggfunc=np.sum)
		pivot = pd.DataFrame(pivot).rename(columns={col:col+'_cat_f'}).reset_index()
		data  = pd.merge(data,pivot,on=['cat_f','time_id'],how='left')
		cat_f_feat.append(col+'_cat_f')
	
	stat_feat = stat_feat + cat_f_feat
	
	
	# cat_son趋势
	cat_son_feat = []
	for col in tend_list:
		pivot = pd.pivot_table(data,index=['cat_son','time_id'],values=col,aggfunc=np.sum)
		pivot = pd.DataFrame(pivot).rename(columns={col:col+'_cat_son'}).reset_index()
		data  = pd.merge(data,pivot,on=['cat_son','time_id'],how='left')
		cat_son_feat.append(col+'_cat_son')
	
	stat_feat = stat_feat + cat_son_feat
	
	
	# 月份（因为日期计数前移至上月25号，所以天数前移一个月）
	count_month = [31,31,28,31,30,31,30,31,31,30,31,30]
	data['count_month'] = list(map(lambda x:count_month[int(x-1)],data['month_id']))
	# 修改闰月天数
	data[data['time_id']==27]['count_month'] = 29
	stat_feat.append('count_month')
	
	
	# 春节月
	#数字需要修改！！！！
	data['is_chunjie'] = list(map(lambda x:1 if x==2 or x==14 or x==25 or x==38 else 0,data['time_id']))
	data['is_chunjie_before'] = list(map(lambda x:1 if x==1 or x==13 or x==24 or x==37 else 0,data['time_id']))
	data['is_chunjie_later']   = list(map(lambda x:1 if x==3 or x==15 or x==26 or x==39 else 0,data['time_id']))
	# 疫情月
	data['covid'] = list(map(lambda x:1 if x==26 else 0,data['time_id']))
	data['covid_after'] = list(map(lambda x:1 if x==27 else 0,data['time_id']))
	month_st_stat_feat = ['is_chunjie','is_chunjie_before','is_chunjie_later','covid','covid_after']
	stat_feat = stat_feat + month_st_stat_feat
	
	return data,stat_feat

In [7]:
def get_model_type():   
	model = lgb.LGBMRegressor(
			num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='mse',
			max_depth=6, learning_rate=0.1, min_child_samples=1, random_state=0,
			n_estimators=2000, subsample=0.8, colsample_bytree=0.6,
			)
	return model

In [8]:
def get_train_model(df_, m, m_type,features, num_feat, cate_feat):
	
	df = df_.copy()
	# 数据集划分
	all_idx   = df['time_id'].between(0 , m-1)
	test_idx  = df['time_id'].between(m , m  )
	#初始化model
	model = get_model_type()
	model.fit(df[all_idx][features], df[all_idx]['label'], categorical_feature=cate_feat,verbose=100)
	df['forecastVolum'] = model.predict(df[features]) 
	df_importance = pd.DataFrame({
		'column': features,
		'importance': model.feature_importances_,
	})
	df_importance = df_importance.sort_values(by='importance',ascending=False).reset_index(drop=True)
	df_importance.to_csv('G:/kaggle竞赛GM之路！！！/竞赛ing/2021工业大数据竞赛-物料需求预测/submit_final/importance.csv',index=False)
	sub = df[test_idx][['id']]
	sub['forecastVolum'] = df[test_idx]['forecastVolum']
	return sub

In [9]:
def LGB(input_data):
	#这里采取分月预测的形式，分别预测39，40，41月
	
	input_data['label'] = list(map(lambda x : x if x==np.NAN else math.log(x+1,lg),input_data['label']))
	input_data['demandVolume'] = list(map(lambda x : x if x==np.NAN else math.log(x+1,lg),input_data['demandVolume']))
	input_data['jidu_id'] = ((input_data['month_id']-1)/3+1).map(int)
	'******************************分月预测************************************************************'
	for month in [39,40,41]:
		m_type = 'lgb'
		data_df, stat_feat = get_stat_feature(input_data,month)
		num_feat = ['sales_year']+stat_feat
		cate_feat = ['id','f_code','m_code','brand','cat_f','cat_son','month_id','jidu_id']
		for i in cate_feat:
			data_df[i] = data_df[i].astype('category')
		features = num_feat + cate_feat
		sub = get_train_model(data_df, month, m_type, features, num_feat, cate_feat)
		input_data.loc[(input_data.time_id==month),  'demandVolume'] = sub['forecastVolum'].values
		input_data.loc[(input_data.time_id==month),  'label'      ] = sub['forecastVolum'].values
	input_data['demandVolume'] = list(map(lambda x : x if x==np.NAN else (lg**(x))-1, input_data['demandVolume']))
	sub = input_data.loc[(input_data.time_id >= 39),['id','time_id','demandVolume']]
	sub.columns = ['id','time_id','forecastVolum']
	sub['id'] = sub['id'].map(int)
	return sub

In [10]:
sub_970=LGB(input_data)

In [11]:
input_data['id'] = input_data['id'].map(int)
input_data = pd.merge(input_data,sub_970,on=['id','time_id'],how='left')
input_data = input_data.loc[input_data.time_id>=39,['f_code','m_code','time_id','forecastVolum']]
input_data = input_data.reset_index(drop=True)

In [12]:
sub_M1 = input_data.loc[input_data.time_id==39,['f_code','m_code','forecastVolum']]
sub_M2 = input_data.loc[input_data.time_id==40,['f_code','m_code','forecastVolum']]
sub_M3 = input_data.loc[input_data.time_id==41,['f_code','m_code','forecastVolum']]

sub_lgb = sub_M1.sort_values(by=['f_code','m_code']).reset_index(drop=True)
sub_lgb.rename(columns={'forecastVolum':'M+1月预测需求量'},inplace=True)

sub_lgb = pd.merge(sub_lgb,sub_M2,on=['f_code','m_code'],how='left')
sub_lgb.rename(columns={'forecastVolum':'M+2月预测需求量'},inplace=True)

sub_lgb = pd.merge(sub_lgb,sub_M3,on=['f_code','m_code'],how='left')
sub_lgb.rename(columns={'forecastVolum':'M+3月预测需求量'},inplace=True)

sub_lgb.rename(columns={'f_code':'工厂编码','m_code':'物料编码'},inplace=True)

In [13]:
sub_lgb.to_csv('G:/kaggle竞赛GM之路！！！/竞赛ing/2021工业大数据竞赛-物料需求预测/submit_final/submission.csv',index=False)